<a href="https://colab.research.google.com/github/sohamtalukdar/Enron-Email-Analysis/blob/main/Vlabs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!mkdir "/content/drive/MyDrive/maildir/"
!tar -xzvf "/content/drive/MyDrive/enron_with_categories.tar.gz" -C "/content/drive/MyDrive/maildir/"

In [91]:
import pandas as pd 
import csv
import os
import nltk
nltk.download('punkt')
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [77]:

import pandas as pd 
import csv
import os
# Map folder names to label values
labels = {
    "1": "Company Business, Strategy, etc.",
    "2": "Purely Personal",
    "3": "Personal but in professional context",
    "4": "Logistic Arrangements",
    "5": "Status arrangements",
    "6": "Document editing/checking",
    "7": "Empty message",
    "8": "Empty message"
}

root_directory = '/content/drive/MyDrive/maildir/enron_with_categories/'
fields = [ "#","Label", "Message"]
rows = []

for folder_name in os.listdir(root_directory):
    folder_path = os.path.join(root_directory, folder_name)
    if os.path.isdir(folder_path):
        label = labels[folder_name]
        for filename in os.listdir(folder_path):
            if filename.endswith(".txt"):
                with open(os.path.join(folder_path, filename), 'r') as f:
                    data = f.read()
                    message_body = data.split("\n\n")[-1]
                    number = int(filename.split(".")[0])
                    rows.append({"Message": message_body, "Label": label, "#": number})

with open("email_bodies.csv", "w", newline="") as f:
    writer = csv.DictWriter(f, fieldnames=fields)
    writer.writeheader()
    for row in rows:
        writer.writerow(row)
 

In [78]:
pd.read_csv("email_bodies.csv")

,#,Label,Message
0,114715,"Company Business, Strategy, etc.",NaN
1,229405,"Company Business, Strategy, etc.",Bonne \nEB1669B
2,232795,"Company Business, Strategy, etc.",NaN
3,62815,"Company Business, Strategy, etc.",Will provide a further report on market-based ...
4,118871,"Company Business, Strategy, etc.","Judge asked everyone to consider how to ""Share..."
...,...,...,...
1697,232322,Empty message,- #92332 v1 - Verification - Interrogatories.wpd
1698,175448,Empty message,\n
1699,175814,Empty message,+---------------------------------------------...
1700,175816,Empty message,\nSue Mara\nEnron Corp.\nTel: (415) 782-7802\n...


In [84]:


# Load the data into a pandas DataFrame
df = pd.read_csv('/content/email_bodies.csv')

# Handle missing values
df.fillna(df.mean(), inplace=True)

# Remove duplicates
df.drop_duplicates(inplace=True)

# Encoding categorical variables
df = pd.get_dummies(df, columns=["Label"])

# Dropping rows with NAN values
df = df.dropna(subset=["Message"])

<ipython-input-84-4b3f556ab8ef>:5: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df.fillna(df.mean(), inplace=True)


In [85]:
df

,#,Message,"Label_Company Business, Strategy, etc.",Label_Document editing/checking,Label_Empty message,Label_Logistic Arrangements,Label_Personal but in professional context,Label_Purely Personal,Label_Status arrangements
1,229405,Bonne \nEB1669B,1,0,0,0,0,0,0
3,62815,Will provide a further report on market-based ...,1,0,0,0,0,0,0
4,118871,"Judge asked everyone to consider how to ""Share...",1,0,0,0,0,0,0
5,106588,Thanks for your attention to this request. Pl...,1,0,0,0,0,0,0
6,122973,Jeff - please put these items together.,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...
1696,125772,\n???,0,0,1,0,0,0,0
1697,232322,- #92332 v1 - Verification - Interrogatories.wpd,0,0,1,0,0,0,0
1698,175448,\n,0,0,1,0,0,0,0
1699,175814,+---------------------------------------------...,0,0,1,0,0,0,0


In [106]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import string
from nltk.tokenize import word_tokenize
import re

nltk.download('stopwords')
stop_words = set(stopwords.words('english'))
stemmer = PorterStemmer()
punctuation = string.punctuation

def normalize_text(text):
    # Lowercase the text
    text = text.lower()
    
    # Remove punctuation
    text = text.translate(str.maketrans("", "", punctuation))
    
    # Remove numbers
    text = re.sub(r'\d+', '', text)
    
    # Remove HTML tags and special characters
    text = re.sub(r'<.*?>|[^\w\s]', '', text)

    # Remove non-ASCII characters
    text = re.sub(r'[^\x00-\x7F]+', '', text)
    
    # Remove email addresses and URLs
    text = re.sub(r'[\w\.-]+@[\w\.-]+|https?://[\w\.-/]+', '', text)
    
    # Remove repeated characters
    text = re.sub(r'(.)\1{2,}', r'\1\1', text)
    
    # Remove extra whitespace
    text = re.sub(r'\s+', ' ', text)
    
    # Tokenize the text
    words = word_tokenize(text)
    
    # Remove stop words
    words = [word for word in words if word not in stop_words]

    # Remove short words
    words = [word for word in words if len(word) > 2]
    
    # Stem the words
    words = [stemmer.stem(word) for word in words]
    
    return " ".join(words)

df["Message"] = df["Message"].apply(normalize_text)


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [107]:
df

,#,Message,"Label_Company Business, Strategy, etc.",Label_Document editing/checking,Label_Empty message,Label_Logistic Arrangements,Label_Personal but in professional context,Label_Purely Personal,Label_Status arrangements
1,229405,bonn ebb,1,0,0,0,0,0,0
3,62815,provid report marketba rate sum leon sympathet...,1,0,0,0,0,0,0
4,118871,judg ask everyon consid share pain thursday me...,1,0,0,0,0,0,0
5,106588,thank attent request plea give call discuss pl...,1,0,0,0,0,0,0
6,122973,jeff plea put item togeth,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...
1696,125772,,0,0,1,0,0,0,0
1697,232322,verif interrogatorieswpd,0,0,1,0,0,0,0
1698,175448,,0,0,1,0,0,0,0
1699,175814,messag may contain confidenti andor privileg i...,0,0,1,0,0,0,0
